### 1. Setting Up

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
import pandas as pd
import numpy as np
import os

# Record Linkage
import recordlinkage as rl
from recordlinkage.index import Block
from recordlinkage.preprocessing import clean

# Regular expression operations
import re

### 2. Preprocessing Data

In [3]:
df = pd.read_excel("data_with_more_match_IDs_2_2_20.xlsx")

In [4]:
df.shape

(50849, 250)

In [5]:
columns_to_drop = [c for c in df.columns if 'Unnamed' in c]

In [6]:
df.drop(columns_to_drop, axis = 1, inplace = True)

In [7]:
print("There are {} columns in df.".format(len(df.columns)))
df.columns

There are 45 columns in df.


Index(['ID', 'Census.Year', 'State/Province_x', 'County_x', 'Place_x',
       'Household Joint ID', 'Joint ID for Matched Records', 'Last.Name',
       'First.Name', 'CalculatedBirthYear', 'Age', 'Sex_x',
       'Color..Race.or.Ethnicity', 'lat_x', 'long_x', 'address', 'MARITAL',
       'WARD', 'ROLL or Sheet#', 'PROFESSION_x', 'Notable_x', 'STREET',
       'PLACEOFBIRTH_x', 'RELIGION_x',
       'NOTE these only apply to narrative answers', 'LIVING W MALE FAMILY?',
       'LIVING W FEMALE FAMILY?', 'LIVING W MALE NONFAMILY?',
       'LIVING W FEMALE NONFAMILY?', 'Cannot Read', 'Cannot Write', 'Sick',
       'Relation to Head of Household',
       'Year of Immigration to Canada if an Immigrant', 'Date of Death',
       'Cause of Death', 'Rank (Military)', 'Enlistment Date',
       'Enlistment Place', 'Date Mustered Out', 'Year of this Record',
       'Last Name MATCH', 'First Name Match', 'Census Year Match',
       'Total of Matches'],
      dtype='object')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50849 entries, 0 to 50848
Data columns (total 45 columns):
ID                                               43561 non-null object
Census.Year                                      50849 non-null object
State/Province_x                                 38866 non-null object
County_x                                         26466 non-null object
Place_x                                          38691 non-null object
Household Joint ID                               406 non-null object
Joint ID for Matched Records                     2235 non-null object
Last.Name                                        50848 non-null object
First.Name                                       50816 non-null object
CalculatedBirthYear                              50839 non-null object
Age                                              50004 non-null object
Sex_x                                            48143 non-null object
Color..Race.or.Ethnicity                  

#### Creating 'unique_id', a unique identifier for the records.

In [9]:
df['unique_id'] = df.index + 1

#### Selecting columns for deduplication

'address' is left out because it includes 'place', 'county' and 'state_or_province'.

In [10]:
df_filtered = df[['ID', 'unique_id', 
                  'Last.Name', 'First.Name',
                  'State/Province_x', 'County_x', 'Place_x', 'WARD', 'STREET', 'PLACEOFBIRTH_x',
                  'CalculatedBirthYear', 'Census.Year' , 'Sex_x', 'Color..Race.or.Ethnicity']]

Standardize the column names.

In [11]:
df_filtered = df_filtered.rename(columns = {'Last.Name': 'last_name',
                                            'First.Name': 'first_name', 
                                            'State/Province_x': 'state_or_province', 
                                            'County_x': 'county', 
                                            'Place_x': 'place', 
                                            'WARD': 'ward', 
                                            'STREET': 'street', 
                                            'PLACEOFBIRTH_x': 'place_of_birth',
                                            'Census.Year': 'census_year', 
                                            'CalculatedBirthYear': 'calculated_birth_year', 
                                            'Sex_x': 'sex', 
                                            'Color..Race.or.Ethnicity': 'race'})

#### Preprocessing Birth Year

In [12]:
df_filtered['calculated_birth_year'].unique()

array([1845, 1822, 1832, 1823, 1841, 1816, 1859, 1857, 1838, 1829, 1860,
       1843, 1846, 1854, 1826, 1837, 1856, 1855, 1824, 1825, 1853, 1844,
       1842, 1840, 1821, 1849, 1858, 1848, 1800, 1831, 1847, 1814, 1852,
       1819, 1851, 1850, 1810, 1820, 1839, 1812, 1836, 1811, 1801, 1817,
       1835, 1828, 1833, 1806, 1805, 1796, 1834, 1807, 1797, 1827, 1808,
       1791, 1830, 1818, 1861, 1862, 1815, 1793, 1813, 1878, 1866, 1877,
       1880, 1865, 1872, 1863, 1879, 1875, 1868, 1864, 1898, '#VALUE!',
       1882, 1899, 1871, 1905, 1780, 1785, 1874, 1888, 1903, 1910, 1894,
       1870, 1895, 1883, 1907, 1891, 1908, 1873, 1876, 1889, 1902, 1887,
       1885, 1884, 1900, 1901, 1802, 1881, 1869, 1786, 1803, 1886, 1867,
       1919, 1795, 1892, 1893, 1904, 1890, 1909, 1804, 1799, 1809, 1787,
       1790, 1798, 1794, 1896, 1897, 1775, 1772, 1788, 1915, 1913, 1783,
       1911, 1792, 1929, '-', 1920, 1781, 1782, 1776, 1914, 1770, 1906,
       nan, 1912, 1918, 1778, 1789, 'F', 'April 26, 1

In [13]:
df_filtered['census_year'].unique()

array([1900, 1910, 1870, 1880, 1861, 1864, 1860, 1920, 1850, 1930, 1851,
       1891, 'Ontario, Canada Voter Lists 1867-1900', 1911, 1881, 1871,
       1872, 'U.S. Civil War Records and Soldier Profiles 1861-1865',
       1901,
       'Canada, City and Area Directories, 1819-1906, 1867 Directory',
       '1855 Massachussetts State Census',
       'Ontario, Canada, Deaths and Deaths Overseas 1869-1947',
       'Find-A-Grave Index', 1865, 'Michigan Marriage Records 1867-1952',
       'New York State Census 1892',
       'Ontario, Canada, Deaths and Deaths Overseas, 1869-1947', 1899,
       'Cook County, Illinois, Deaths Index, 1878-1932',
       'Virginia, Death Records 1912-2014',
       'Canada, City and Area Directories, 1819-1906'], dtype=object)

In [14]:
def prc_year(row, year_col):
    """
    Processing birth year.
    If birth year is not a number, not in a date format, or is noncompliant,
    it will be converted to NaN.
    """
    
    # Helper functions
    def is_number(num):
        try:
            float(num)
            return True
        except ValueError:
            return False
    
    non_compliant_values = ['-', 'F', '#VALUE!']
    if row[year_col] in non_compliant_values or pd.isnull(row[year_col]):
        return np.nan
    elif is_number(row[year_col]):
        return float(row[year_col])
    elif re.findall('\d{4}', row[year_col]):
        return min([float(i) for i in re.findall('\d{4}', row[year_col])])
    else:
        return np.nan

In [15]:
df_filtered['calculated_birth_year'] = df_filtered.apply(lambda row: prc_year(row, 'calculated_birth_year'),
                                                         axis = 1)

In [16]:
df_filtered['census_year'] = df_filtered.apply(lambda row: prc_year(row, 'census_year'),
                                                         axis = 1)

In [17]:
print('There are {0} unique birth year, from {1} to {2}.'.format(len(df_filtered['calculated_birth_year'].unique()),
                                                                 min(df_filtered['calculated_birth_year']),
                                                                 max(df_filtered['calculated_birth_year'])))
sorted(df_filtered['calculated_birth_year'].unique())

There are 162 unique birth year, from 1757.0 to 1930.0.


[1780.0,
 1785.0,
 1791.0,
 1793.0,
 1796.0,
 1797.0,
 1800.0,
 1801.0,
 1805.0,
 1806.0,
 1807.0,
 1808.0,
 1810.0,
 1811.0,
 1812.0,
 1813.0,
 1814.0,
 1815.0,
 1816.0,
 1817.0,
 1818.0,
 1819.0,
 1820.0,
 1821.0,
 1822.0,
 1823.0,
 1824.0,
 1825.0,
 1826.0,
 1827.0,
 1828.0,
 1829.0,
 1830.0,
 1831.0,
 1832.0,
 1833.0,
 1834.0,
 1835.0,
 1836.0,
 1837.0,
 1838.0,
 1839.0,
 1840.0,
 1841.0,
 1842.0,
 1843.0,
 1844.0,
 1845.0,
 1846.0,
 1847.0,
 1848.0,
 1849.0,
 1850.0,
 1851.0,
 1852.0,
 1853.0,
 1854.0,
 1855.0,
 1856.0,
 1857.0,
 1858.0,
 1859.0,
 1860.0,
 1861.0,
 1862.0,
 1863.0,
 1864.0,
 1865.0,
 1866.0,
 1868.0,
 1871.0,
 1872.0,
 1875.0,
 1877.0,
 1878.0,
 1879.0,
 1880.0,
 1882.0,
 1898.0,
 nan,
 1757.0,
 1759.0,
 1760.0,
 1762.0,
 1764.0,
 1765.0,
 1770.0,
 1771.0,
 1772.0,
 1773.0,
 1774.0,
 1775.0,
 1776.0,
 1777.0,
 1778.0,
 1779.0,
 1781.0,
 1782.0,
 1783.0,
 1784.0,
 1786.0,
 1787.0,
 1788.0,
 1789.0,
 1790.0,
 1792.0,
 1794.0,
 1795.0,
 1798.0,
 1799.0,
 1802.0,
 180

In [18]:
print('There are {0} unique census year, from {1} to {2}.'.format(len(df_filtered['census_year'].unique()),
                                                                  min(df_filtered['census_year']),
                                                                  max(df_filtered['census_year'])))
sorted(df_filtered['census_year'].unique())

There are 27 unique census year, from 1819.0 to 1930.0.


[1819.0,
 1850.0,
 1851.0,
 1855.0,
 1860.0,
 1861.0,
 1864.0,
 1865.0,
 1867.0,
 1869.0,
 1870.0,
 1871.0,
 1872.0,
 1878.0,
 1880.0,
 1881.0,
 1891.0,
 1892.0,
 1899.0,
 1900.0,
 1901.0,
 1910.0,
 1911.0,
 1912.0,
 1920.0,
 1930.0,
 nan]

#### Preprocessing State or Province

In [19]:
df_filtered['state_or_province'].unique()

array(['ME', 'NY', 'Ontario', 'Canada West', 'CanadaWest', nan, 'Ohio',
       'MI', 'RI', 'CA', 'OR', 'MA', 'WV', 'IL', 'PA', 'SC', 'WA', 'OH',
       'IA', 'MN', 'NJ', 'VT', 'GA', 'TX', 'HI', 'DC', 'TN', 'LA', 'OK',
       'MO', 'AL', 'KY', 'FL', 'CT', 'MS', 'NE', 'AR', 'NM', 'MD', 'IN',
       'VA', 'NC', 'MT', 'Vermont', 'WI', 'WY', 'NH', 'DE',
       'Pennsylvania', 'CO', 'NV', 'SD', 'KS', 'Illinois', 'PANA', 'ID',
       'ND', 'VI', 'toledo', 'MIL', 'AZ', 'AK', 'Michigan', 'UT', 'PR',
       'Massachussetts', 'District of Columbia', 'New York', 'Wisconsin',
       'Canada West (Ontario)', 'Alabama', 'ITER', 'Virginia'],
      dtype=object)

In [20]:
def transform_state(row):    
    """
    This function uses a dictionary
    to covert historical or non-abbreviated states or provinces
    into abbreviated form. 
    An important notice is that Canada West will all be converted into ON (Ontario).
    Unknown values: PANA, ITER, MIL.
    """
    state_dict = {'CanadaWest': 'ON',
                  'Ontario': 'ON',
                  'Canada West': 'ON',
                  'Pennsylvania': 'PA',
                  'Illinois': 'IL',
                  'Wisconsin': 'WI',
                  'District of Columbia': 'DC',
                  'Alabama': 'AL',
                  'Vermont': 'VT',
                  'Michigan': 'MI',
                  'Ohio': 'OH',
                  'Massachussetts': 'MA',
                  'Virginia': 'VA',
                  'Canada West (Ontario)': 'ON',
                  'New York': 'NY',
                  'toledo': 'OH'}
    
    if row['state_or_province'] in state_dict.keys():
        return state_dict[row['state_or_province']]
    else:
        return row['state_or_province']

df_filtered['state_or_province'] = df_filtered.apply(lambda row: transform_state(row), axis = 1) 

In [21]:
print('There are {} states or provinces.'.format(len(df_filtered['state_or_province'].unique())))
df_filtered['state_or_province'].unique()

There are 58 states or provinces.


array(['ME', 'NY', 'ON', nan, 'OH', 'MI', 'RI', 'CA', 'OR', 'MA', 'WV',
       'IL', 'PA', 'SC', 'WA', 'IA', 'MN', 'NJ', 'VT', 'GA', 'TX', 'HI',
       'DC', 'TN', 'LA', 'OK', 'MO', 'AL', 'KY', 'FL', 'CT', 'MS', 'NE',
       'AR', 'NM', 'MD', 'IN', 'VA', 'NC', 'MT', 'WI', 'WY', 'NH', 'DE',
       'CO', 'NV', 'SD', 'KS', 'PANA', 'ID', 'ND', 'VI', 'MIL', 'AZ',
       'AK', 'UT', 'PR', 'ITER'], dtype=object)

In [22]:
def transform_race(row):    
    
    if pd.notnull(row['race']):
        row['race'] = row['race'].lower()
        row['race'] = row['race'].replace('\xa0', '')
    
    race_dict = {'mulatto(blackandwhite)': 'MIXED',
                 'm(wonancestry.com)': 'MIXED',
                 'mulatto': 'MIXED',
                 'mullato': 'MIXED',
                 'm': 'MIXED',
                 'm(winancestry.com)': 'MIXED',
                 'black': 'BLACK',
                 'b': 'BLACK',
                 'blk': 'BLACK',
                 'african': 'BLACK',
                 'dark': 'BLACK',
                 'drk': 'BLACK',
                 'african (black)': 'BLACK',
                 '“negro”': 'BLACK',
                 'negro': 'BLACK',
                 'blacj': 'BLACK', 
                 'bkj': 'BLACK', 
                 'white': 'WHITE',
                 'w': 'WHITE',
                 '[w]': 'WHITE',
                 'white': 'WHITE',
                 'white in black household': 'WHITE',
                 'white but passing': 'WHITE',
                 'ancestrysaysw': 'WHITE'}
    
    if row['race'] in race_dict.keys():
        return race_dict[row['race']]
    elif pd.notnull(row['race']):
        return 'OTHERS'
    else:
        return np.nan

df_filtered['race'] = df_filtered.apply(lambda row: transform_race(row), axis = 1)

In [23]:
df_filtered['race'].unique()

array(['BLACK', 'MIXED', 'WHITE', 'OTHERS', nan], dtype=object)

In [24]:
column_to_clean = ['county',
                   'place',
                   'last_name',
                   'first_name',
                   'race',
                   'sex',
                   'place_of_birth']
for i in column_to_clean:
    df_filtered[i] = clean(df_filtered[i])

In [25]:
df_filtered.head(5)

,ID,unique_id,last_name,first_name,state_or_province,county,place,ward,street,place_of_birth,calculated_birth_year,census_year,sex,race
0,9839,1,cornelison,albert,ME,aroostook,smyrna,NaN,NaN,NaN,1845.0,1900.0,m,black
1,15009,2,cornelison,marye,ME,aroostook,houlton,NaN,NaN,NaN,1845.0,1910.0,f,black
2,1277,3,dam,david,NY,franklin,chateaugay,NaN,NaN,NaN,1822.0,1870.0,m,mixed
3,1278,4,dam,margratte,NY,franklin,chateaugay,NaN,NaN,NaN,1822.0,1870.0,f,white
4,3686,5,fields,maryann,NY,erie,4 wdbuffalo,NaN,NaN,NaN,1832.0,1880.0,f,black


#### Create a "full_name" column for record linkage.

In [26]:
df_filtered['full_name'] = df_filtered['first_name'] + ' ' + df_filtered['last_name']

### 3. Deduplication - Strictest

Two records are duplications when:
* 'state_or_province', 'county', 'place', 'ward', 'street' are almost the same.
* 'last_name', 'first_name' are the same.
* 'calculated_birth_year' is the same.
* 'sex' is the same.
* 'race' is the same.
* 'place_of_birth' is left out because it is too messy.

In [59]:
df1 = df_filtered
df2 = df_filtered

In [60]:
indexer = rl.Index()
indexer.add(Block('full_name', 'full_name'))
record_links = indexer.index(df1, df2)

In [61]:
print(len(record_links))

121613


In [62]:
comparer = rl.Compare()
comparer.string('first_name', 'first_name', method = 'jarowinkler', threshold = 0.9, label = 'first_name')
comparer.string('last_name', 'last_name', method = 'jarowinkler', threshold = 0.9, label = 'last_name')
comparer.string('state_or_province', 'state_or_province', method = 'jarowinkler', threshold = 0.9, label = 'state')
comparer.string('county', 'county', method = 'jarowinkler', threshold = 0.9, label = 'county')
comparer.string('place', 'place', method = 'jarowinkler', threshold = 0.9, label = 'place')
comparer.exact('race', 'race', label = 'race')
comparer.exact('sex', 'sex', label = 'sex')
comparer.exact('calculated_birth_year', 'calculated_birth_year', label = 'calculated_birth_year')
comparer.exact('census_year', 'census_year', label = 'census_year')

<Compare>

In [63]:
compare_vectors_rl = comparer.compute(record_links, df1, df2)
compare_vectors_rl[0:20]

first_name  last_name  state  county  place  race  sex  \
0     0             1.0        1.0    1.0     1.0    1.0     1    1   
      9979          1.0        1.0    0.0     0.0    0.0     1    1   
9979  0             1.0        1.0    0.0     0.0    0.0     1    1   
      9979          1.0        1.0    0.0     0.0    0.0     1    1   
1     1             1.0        1.0    1.0     1.0    1.0     1    1   
      9967          1.0        1.0    0.0     0.0    0.0     1    1   
9967  1             1.0        1.0    0.0     0.0    0.0     1    1   
      9967          1.0        1.0    0.0     0.0    0.0     1    1   
2     2             1.0        1.0    1.0     1.0    1.0     1    1   
      11115         1.0        1.0    1.0     1.0    1.0     0    1   
11115 2             1.0        1.0    1.0     1.0    1.0     0    1   
      11115         1.0        1.0    1.0     1.0    1.0     1    1   
3     3             1.0        1.0    1.0     1.0    1.0     1    1   
4     4             1.0        1.0    1.0     1.0    1.0     1    1   
      14662         1.0        1.0    1.0     0.0    0.0     1    1   
14662 4             1.0        1.0    1.0     0.0    0.0     1    1   
      14662         1.0        1.0    1.0     0.0    1.0     1    1   
5     5             1.0        1.0    1.0     1.0    1.0     1    1   
      14640         1.0        1.0    1.0     0.0    0.0     1    1   
      14663         1.0        1.0    1.0     0.0    0.0     1    1   

             calculated_birth_year  census_year  
0     0                          1            1  
      9979                       0            0  
9979  0                          0            0  
      9979                       1            1  
1     1                          1            1  
      9967                       0            0  
9967  1                          0            0  
      9967                       1            1  
2     2                          1            1  
      11115                      0            1  
11115 2                          0            1  
      11115                      1            1  
3     3                          1            1  
4     4                          1            1  
      14662                      1            1  
14662 4                          1            1  
      14662                      1            1  
5     5                          1            1  
      14640                      0            0  
      14663                      0            0

In [64]:
result_rl = compare_vectors_rl[(compare_vectors_rl['first_name'] == 1.0) & 
                               (compare_vectors_rl['last_name'] == 1.0) &
                               (compare_vectors_rl['state'] == 1.0) &
                               (compare_vectors_rl['county'] == 1.0) &
                               (compare_vectors_rl['place'] == 1.0) &
                               (compare_vectors_rl['sex'] == 1) &
                               (compare_vectors_rl['race'] == 1) &
                               (compare_vectors_rl['census_year'] == 1) &
                               (compare_vectors_rl['calculated_birth_year'] == 1)].\
                                reset_index()
result_rl = result_rl[result_rl['level_0'] != result_rl['level_1']].reset_index()
result_rl.drop('index', axis = 1, inplace = True)

In [65]:
result_rl

,level_0,level_1,first_name,last_name,state,county,place,race,sex,calculated_birth_year,census_year
0,795,826,1.0,1.0,1.0,1.0,1.0,1,1,1,1
1,826,795,1.0,1.0,1.0,1.0,1.0,1,1,1,1
2,1686,1694,1.0,1.0,1.0,1.0,1.0,1,1,1,1
3,1694,1686,1.0,1.0,1.0,1.0,1.0,1,1,1,1
4,1827,1828,1.0,1.0,1.0,1.0,1.0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
223,49251,49250,1.0,1.0,1.0,1.0,1.0,1,1,1,1
224,50036,50037,1.0,1.0,1.0,1.0,1.0,1,1,1,1
225,50037,50036,1.0,1.0,1.0,1.0,1.0,1,1,1,1
226,50749,50750,1.0,1.0,1.0,1.0,1.0,1,1,1,1


In [66]:
def create_indexid(row):
    return "".join(sorted([str(int(i)) for i in [row['level_0'], row['level_1']]]))
result_rl['indexid'] = result_rl.apply(lambda row: create_indexid(row), axis = 1)
result_rl = result_rl.drop_duplicates('indexid')

In [67]:
result_rl

,level_0,level_1,first_name,last_name,state,county,place,race,sex,calculated_birth_year,census_year,indexid
0,795,826,1.0,1.0,1.0,1.0,1.0,1,1,1,1,795826
2,1686,1694,1.0,1.0,1.0,1.0,1.0,1,1,1,1,16861694
4,1827,1828,1.0,1.0,1.0,1.0,1.0,1,1,1,1,18271828
6,2081,2083,1.0,1.0,1.0,1.0,1.0,1,1,1,1,20812083
8,2082,2084,1.0,1.0,1.0,1.0,1.0,1,1,1,1,20822084
...,...,...,...,...,...,...,...,...,...,...,...,...
218,47763,47781,1.0,1.0,1.0,1.0,1.0,1,1,1,1,4776347781
220,47764,47782,1.0,1.0,1.0,1.0,1.0,1,1,1,1,4776447782
222,49250,49251,1.0,1.0,1.0,1.0,1.0,1,1,1,1,4925049251
224,50036,50037,1.0,1.0,1.0,1.0,1.0,1,1,1,1,5003650037


In [68]:
df_result_rl = pd.DataFrame()
for i in zip(result_rl['level_0'], result_rl['level_1']):
    df_result_rl = df_result_rl.append(df.iloc[i[0]])
    df_result_rl = df_result_rl.append(df.iloc[i[1]])   
df_result_rl.reset_index(inplace = True)
df_result_rl = df_result_rl.drop('index', axis = 1)

In [69]:
num_list = []
for i in range(int(df_result_rl.shape[0]/2)):
    num_list.append(i)
    num_list.append(i)
df_result_rl['dup_pair'] = pd.Series(num_list)

In [70]:
df_result_rl

,Age,CalculatedBirthYear,Cannot Read,Cannot Write,Cause of Death,Census Year Match,Census.Year,Color..Race.or.Ethnicity,County_x,Date Mustered Out,...,State/Province_x,Total of Matches,WARD,Year of Immigration to Canada if an Immigrant,Year of this Record,address,lat_x,long_x,unique_id,dup_pair
0,26.0,1835.0,NaN,NaN,NaN,NaN,1861.0,M,Hamilton,NaN,...,Ontario,NaN,NaN,NaN,NaN,NaN,NaN,NaN,796.0,0
1,26.0,1835.0,NaN,NaN,NaN,100.0,1861.0,M,Hamilton,NaN,...,Ontario,#REF!,NaN,NaN,NaN,NaN,NaN,NaN,827.0,0
2,27.0,1834.0,NaN,NaN,NaN,NaN,1861.0,B,Hamilton,NaN,...,Ontario,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1687.0,1
3,27.0,1834.0,NaN,NaN,NaN,0.0,1861.0,B,Hamilton,NaN,...,Ontario,#REF!,NaN,NaN,NaN,NaN,NaN,NaN,1695.0,1
4,24.0,1837.0,NaN,NaN,NaN,NaN,1861.0,B,Hamilton,NaN,...,Ontario,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1828.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,23.0,1897.0,NaN,NaN,NaN,NaN,1920.0,B,KINGS,NaN,...,NY,NaN,NaN,NaN,NaN,"BROOKLYN,KINGS,NY",40.6782,-73.94415790000001,49252.0,111
224,28.0,1892.0,NaN,NaN,NaN,NaN,1920.0,B,PENOBSCOT,NaN,...,ME,NaN,NaN,NaN,NaN,"BANGOR,PENOBSCOT,ME",44.8012,-68.77781379999999,50037.0,112
225,28.0,1892.0,NaN,NaN,NaN,NaN,1920.0,B,PENOBSCOT,NaN,...,ME,NaN,NaN,NaN,NaN,"BANGOR,PENOBSCOT,ME",44.801612799999994,-68.7712,50038.0,112
226,35.0,1895.0,NaN,NaN,NaN,NaN,1930.0,B,NEWCASTLE,NaN,...,DE,NaN,NaN,NaN,NaN,"WILMINGTON,NEWCASTLE,DE",39.66203970000001,-75.5668,50750.0,113


In [73]:
df_result_rl.to_csv('duplications.csv')

### 4. Relax 'place'.

In [71]:
result_rl_2 = compare_vectors_rl[(compare_vectors_rl['first_name'] == 1.0) & 
                                 (compare_vectors_rl['last_name'] == 1.0) &
                                 (compare_vectors_rl['state'] == 1.0) &
                                 (compare_vectors_rl['county'] == 1.0) &
                                 # (compare_vectors_rl['place'] == 1.0) &
                                 (compare_vectors_rl['sex'] == 1) &
                                 (compare_vectors_rl['race'] == 1) &
                                 (compare_vectors_rl['census_year'] == 1) &
                                 (compare_vectors_rl['calculated_birth_year'] == 1)].\
                                 reset_index()
result_rl_2 = result_rl_2[result_rl_2['level_0'] != result_rl_2['level_1']].reset_index()
result_rl_2.drop('index', axis = 1, inplace = True)

In [72]:
result_rl_2

,level_0,level_1,first_name,last_name,state,county,place,race,sex,calculated_birth_year,census_year
0,296,17740,1.0,1.0,1.0,1.0,0.0,1,1,1,1
1,17740,296,1.0,1.0,1.0,1.0,0.0,1,1,1,1
2,795,826,1.0,1.0,1.0,1.0,1.0,1,1,1,1
3,826,795,1.0,1.0,1.0,1.0,1.0,1,1,1,1
4,1686,1694,1.0,1.0,1.0,1.0,1.0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
233,49251,49250,1.0,1.0,1.0,1.0,1.0,1,1,1,1
234,50036,50037,1.0,1.0,1.0,1.0,1.0,1,1,1,1
235,50037,50036,1.0,1.0,1.0,1.0,1.0,1,1,1,1
236,50749,50750,1.0,1.0,1.0,1.0,1.0,1,1,1,1


In [73]:
result_rl_2['indexid'] = result_rl_2.apply(lambda row: create_indexid(row), axis = 1)
result_rl_2 = result_rl_2.drop_duplicates('indexid')

In [74]:
result_rl_2

,level_0,level_1,first_name,last_name,state,county,place,race,sex,calculated_birth_year,census_year,indexid
0,296,17740,1.0,1.0,1.0,1.0,0.0,1,1,1,1,17740296
2,795,826,1.0,1.0,1.0,1.0,1.0,1,1,1,1,795826
4,1686,1694,1.0,1.0,1.0,1.0,1.0,1,1,1,1,16861694
6,1827,1828,1.0,1.0,1.0,1.0,1.0,1,1,1,1,18271828
8,2081,2083,1.0,1.0,1.0,1.0,1.0,1,1,1,1,20812083
...,...,...,...,...,...,...,...,...,...,...,...,...
228,47763,47781,1.0,1.0,1.0,1.0,1.0,1,1,1,1,4776347781
230,47764,47782,1.0,1.0,1.0,1.0,1.0,1,1,1,1,4776447782
232,49250,49251,1.0,1.0,1.0,1.0,1.0,1,1,1,1,4925049251
234,50036,50037,1.0,1.0,1.0,1.0,1.0,1,1,1,1,5003650037


In [79]:
diff_2 = set(result_rl_2['indexid']).difference(set(result_rl['indexid']))

In [81]:
delt_2 = result_rl_2[result_rl_2['indexid'].isin(diff_2)]

In [82]:
delt_2

,level_0,level_1,first_name,last_name,state,county,place,race,sex,calculated_birth_year,census_year,indexid
0,296,17740,1.0,1.0,1.0,1.0,0.0,1,1,1,1,17740296
52,11089,11098,1.0,1.0,1.0,1.0,0.0,1,1,1,1,1108911098
76,14738,14740,1.0,1.0,1.0,1.0,0.0,1,1,1,1,1473814740
88,18868,18869,1.0,1.0,1.0,1.0,0.0,1,1,1,1,1886818869
98,20379,20440,1.0,1.0,1.0,1.0,0.0,1,1,1,1,2037920440


In [83]:
df_result_rl_2 = pd.DataFrame()
for i in zip(delt_2['level_0'], delt_2['level_1']):
    df_result_rl_2 = df_result_rl_2.append(df.iloc[i[0]])
    df_result_rl_2 = df_result_rl_2.append(df.iloc[i[1]])   
df_result_rl_2.reset_index(inplace = True)
df_result_rl_2 = df_result_rl_2.drop('index', axis = 1)

In [84]:
num_list = []
for i in range(int(df_result_rl_2.shape[0]/2)):
    num_list.append(i)
    num_list.append(i)
df_result_rl_2['dup_pair'] = pd.Series(num_list)

In [85]:
df_result_rl_2

,Age,CalculatedBirthYear,Cannot Read,Cannot Write,Cause of Death,Census Year Match,Census.Year,Color..Race.or.Ethnicity,County_x,Date Mustered Out,...,State/Province_x,Total of Matches,WARD,Year of Immigration to Canada if an Immigrant,Year of this Record,address,lat_x,long_x,unique_id,dup_pair
0,50.0,1811.0,NaN,NaN,NaN,NaN,1861.0,black,London,NaN,...,Ontario,NaN,NaN,NaN,NaN,NaN,NaN,NaN,297.0,0
1,50.0,1811.0,NaN,NaN,NaN,NaN,1861.0,B,London,NaN,...,Ontario,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17741.0,0
2,21.0,1849.0,NaN,NaN,NaN,NaN,1870.0,Black,Wayne,NaN,...,Michigan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11090.0,1
3,21.0,1849.0,NaN,NaN,NaN,NaN,1870.0,B,WAYNE,NaN,...,MI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11099.0,1
4,20.0,1860.0,1.0,1.0,2.0,NaN,1880.0,Black,Westchester,NaN,...,NY,NaN,0.0,0.0,NaN,5,NaN,6.857901e+09,14739.0,2
5,20.0,1860.0,NaN,NaN,NaN,NaN,1880.0,B,WESTCHESTER,NaN,...,NY,NaN,NaN,NaN,NaN,"3-WD;YONKERS,WESTCHESTER,NY",40.94146729999999,-7.389744e+01,14741.0,2
6,54.0,1866.0,NaN,NaN,NaN,NaN,1920.0,B,AITKIN,NaN,...,MN,NaN,NaN,NaN,NaN,"AITKIN,AITKIN,MN",46.533012899999996,-9.371025e+01,18869.0,3
7,54.0,1866.0,NaN,NaN,NaN,NaN,1920.0,B,AITKIN,NaN,...,MN,NaN,NaN,NaN,NaN,"FLEMING,AITKIN,MN",46.533012899999996,-9.371025e+01,18870.0,3
8,35.0,1835.0,0.0,0.0,1.0,NaN,1870.0,Black,WAYNE,NaN,...,MI,NaN,0.0,0.0,NaN,3,42.3314,-8.304575e+01,20380.0,4
9,35.0,1835.0,NaN,NaN,NaN,NaN,1870.0,B,WAYNE,NaN,...,MI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20441.0,4


In [86]:
df_result_rl_2.to_csv('duplications_2.csv')

### 5. Relax 'place' and 'county'.

In [87]:
result_rl_3 = compare_vectors_rl[(compare_vectors_rl['first_name'] == 1.0) & 
                                 (compare_vectors_rl['last_name'] == 1.0) &
                                 (compare_vectors_rl['state'] == 1.0) &
                                 # (compare_vectors_rl['county'] == 1.0) &
                                 # (compare_vectors_rl['place'] == 1.0) &
                                 (compare_vectors_rl['sex'] == 1) &
                                 (compare_vectors_rl['race'] == 1) &
                                 (compare_vectors_rl['census_year'] == 1) &
                                 (compare_vectors_rl['calculated_birth_year'] == 1)].\
                                 reset_index()
result_rl_3 = result_rl_3[result_rl_3['level_0'] != result_rl_3['level_1']].reset_index()
result_rl_3.drop('index', axis = 1, inplace = True)

In [88]:
result_rl_3

,level_0,level_1,first_name,last_name,state,county,place,race,sex,calculated_birth_year,census_year
0,4,14662,1.0,1.0,1.0,0.0,0.0,1,1,1,1
1,14662,4,1.0,1.0,1.0,0.0,0.0,1,1,1,1
2,14663,14694,1.0,1.0,1.0,0.0,0.0,1,1,1,1
3,14694,14663,1.0,1.0,1.0,0.0,0.0,1,1,1,1
4,18796,18797,1.0,1.0,1.0,0.0,1.0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
1085,50767,50701,1.0,1.0,1.0,0.0,0.0,1,1,1,1
1086,50717,50790,1.0,1.0,1.0,0.0,0.0,1,1,1,1
1087,50790,50717,1.0,1.0,1.0,0.0,0.0,1,1,1,1
1088,50749,50750,1.0,1.0,1.0,1.0,1.0,1,1,1,1


In [89]:
result_rl_3['indexid'] = result_rl_3.apply(lambda row: create_indexid(row), axis = 1)
result_rl_3 = result_rl_3.drop_duplicates('indexid')

In [90]:
diff_3 = set(result_rl_3['indexid']).difference(set(result_rl['indexid']))

In [91]:
delt_3 = result_rl_3[result_rl_3['indexid'].isin(diff_3)]

In [110]:
delt_3

,level_0,level_1,first_name,last_name,state,county,place,race,sex,calculated_birth_year,census_year,indexid
0,4,14662,1.0,1.0,1.0,0.0,0.0,1,1,1,1,146624
2,14663,14694,1.0,1.0,1.0,0.0,0.0,1,1,1,1,1466314694
4,18796,18797,1.0,1.0,1.0,0.0,1.0,1,1,1,1,1879618797
6,23466,23467,1.0,1.0,1.0,0.0,0.0,1,1,1,1,2346623467
8,50395,50479,1.0,1.0,1.0,0.0,0.0,1,1,1,1,5039550479
...,...,...,...,...,...,...,...,...,...,...,...,...
1077,50675,50791,1.0,1.0,1.0,0.0,0.0,1,1,1,1,5067550791
1079,50676,50791,1.0,1.0,1.0,0.0,0.0,1,1,1,1,5067650791
1082,50682,50754,1.0,1.0,1.0,0.0,1.0,1,1,1,1,5068250754
1084,50701,50767,1.0,1.0,1.0,0.0,0.0,1,1,1,1,5070150767


In [92]:
df_result_rl_3 = pd.DataFrame()
for i in zip(delt_3['level_0'], delt_3['level_1']):
    df_result_rl_3 = df_result_rl_3.append(df.iloc[i[0]])
    df_result_rl_3 = df_result_rl_3.append(df.iloc[i[1]])   
df_result_rl_3.reset_index(inplace = True)
df_result_rl_3 = df_result_rl_3.drop('index', axis = 1)

In [93]:
num_list = []
for i in range(int(df_result_rl_3.shape[0]/2)):
    num_list.append(i)
    num_list.append(i)
df_result_rl_3['dup_pair'] = pd.Series(num_list)

In [94]:
df_result_rl_3

,Age,CalculatedBirthYear,Cannot Read,Cannot Write,Cause of Death,Census Year Match,Census.Year,Color..Race.or.Ethnicity,County_x,Date Mustered Out,...,State/Province_x,Total of Matches,WARD,Year of Immigration to Canada if an Immigrant,Year of this Record,address,lat_x,long_x,unique_id,dup_pair
0,48.0,1832.0,NaN,NaN,NaN,NaN,1880.0,B,ERIE,NaN,...,NY,NaN,NaN,NaN,NaN,"4-WD;BUFFALO,ERIE,NY",42.9023,-78.6338,5.0,0
1,48.0,1832.0,1.0,0.0,1.0,NaN,1880.0,Black,NaN,NaN,...,NY,NaN,0.0,3.0,NaN,6,NaN,5.90975e+09,14663.0,0
2,46.0,1834.0,0.0,0.0,4.0,NaN,1880.0,Black,NaN,NaN,...,NY,NaN,0.0,1.0,NaN,6,NaN,5.90975e+09,14664.0,1
3,46.0,1834.0,NaN,NaN,NaN,NaN,1880.0,B,ERIE,NaN,...,NY,NaN,NaN,NaN,NaN,"4-WD;BUFFALO,ERIE,NY",42.9023,-78.6338,14695.0,1
4,11.0,1869.0,0.0,0.0,4.0,NaN,1880.0,Black,NaN,NaN,...,IL,NaN,0.0,0.0,NaN,9,NaN,1.37068e+09,18797.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,30.0,1850.0,NaN,NaN,NaN,NaN,1880.0,B,SUFFOLK,NaN,...,MA,NaN,NaN,NaN,NaN,"BOSTON,SUFFOLK,MA",42.360082500000004,-71.0589,50755.0,428
858,33.0,1847.0,3.0,2.0,1.0,NaN,1880.0,Mulatto(BlackandWhite),NaN,NaN,...,NY,NaN,0.0,0.0,NaN,5,NaN,6090789000,50702.0,429
859,33.0,1847.0,NaN,NaN,NaN,NaN,1880.0,M,KINGS,NaN,...,NY,NaN,NaN,NaN,NaN,"19-WD;BROOKLYN,KINGS,NY",40.7271,-73.9368,50768.0,429
860,5.0,1875.0,0.0,0.0,3.0,NaN,1880.0,Mulatto(BlackandWhite),NaN,NaN,...,NY,NaN,0.0,2.0,NaN,5,NaN,6090789000,50718.0,430


In [95]:
df_result_rl_3.to_csv('duplications_3.csv')

### 6. Relax 'race'.

In [96]:
result_rl_4 = compare_vectors_rl[(compare_vectors_rl['first_name'] == 1.0) & 
                                 (compare_vectors_rl['last_name'] == 1.0) &
                                 (compare_vectors_rl['state'] == 1.0) &
                                 (compare_vectors_rl['county'] == 1.0) &
                                 (compare_vectors_rl['place'] == 1.0) &
                                 (compare_vectors_rl['sex'] == 1) &
                                 # (compare_vectors_rl['race'] == 1) &
                                 (compare_vectors_rl['census_year'] == 1) &
                                 (compare_vectors_rl['calculated_birth_year'] == 1)].\
                                 reset_index()
result_rl_4 = result_rl_4[result_rl_4['level_0'] != result_rl_4['level_1']].reset_index()
result_rl_4.drop('index', axis = 1, inplace = True)

In [97]:
result_rl_4

,level_0,level_1,first_name,last_name,state,county,place,race,sex,calculated_birth_year,census_year
0,795,826,1.0,1.0,1.0,1.0,1.0,1,1,1,1
1,826,795,1.0,1.0,1.0,1.0,1.0,1,1,1,1
2,1686,1694,1.0,1.0,1.0,1.0,1.0,1,1,1,1
3,1694,1686,1.0,1.0,1.0,1.0,1.0,1,1,1,1
4,1827,1828,1.0,1.0,1.0,1.0,1.0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
235,49350,49349,1.0,1.0,1.0,1.0,1.0,0,1,1,1
236,50036,50037,1.0,1.0,1.0,1.0,1.0,1,1,1,1
237,50037,50036,1.0,1.0,1.0,1.0,1.0,1,1,1,1
238,50749,50750,1.0,1.0,1.0,1.0,1.0,1,1,1,1


In [98]:
result_rl_4['indexid'] = result_rl_4.apply(lambda row: create_indexid(row), axis = 1)
result_rl_4 = result_rl_4.drop_duplicates('indexid')

In [99]:
diff_4 = set(result_rl_4['indexid']).difference(set(result_rl['indexid']))

In [100]:
delt_4 = result_rl_4[result_rl_4['indexid'].isin(diff_4)]

In [115]:
delt_4

,level_0,level_1,first_name,last_name,state,county,place,race,sex,calculated_birth_year,census_year,indexid
20,5793,6102,1.0,1.0,1.0,1.0,1.0,0,1,1,1,57936102
72,14104,14107,1.0,1.0,1.0,1.0,1.0,0,1,1,1,1410414107
110,24941,24942,1.0,1.0,1.0,1.0,1.0,0,1,1,1,2494124942
188,35751,35752,1.0,1.0,1.0,1.0,1.0,0,1,1,1,3575135752
200,39155,39156,1.0,1.0,1.0,1.0,1.0,0,1,1,1,3915539156
234,49349,49350,1.0,1.0,1.0,1.0,1.0,0,1,1,1,4934949350


In [101]:
df_result_rl_4 = pd.DataFrame()
for i in zip(delt_4['level_0'], delt_4['level_1']):
    df_result_rl_4 = df_result_rl_4.append(df.iloc[i[0]])
    df_result_rl_4 = df_result_rl_4.append(df.iloc[i[1]])   
df_result_rl_4.reset_index(inplace = True)
df_result_rl_4 = df_result_rl_4.drop('index', axis = 1)

In [102]:
num_list = []
for i in range(int(df_result_rl_4.shape[0]/2)):
    num_list.append(i)
    num_list.append(i)
df_result_rl_4['dup_pair'] = pd.Series(num_list)

In [103]:
df_result_rl_4

,Age,CalculatedBirthYear,Cannot Read,Cannot Write,Cause of Death,Census Year Match,Census.Year,Color..Race.or.Ethnicity,County_x,Date Mustered Out,...,State/Province_x,Total of Matches,WARD,Year of Immigration to Canada if an Immigrant,Year of this Record,address,lat_x,long_x,unique_id,dup_pair
0,35.0,1826.0,NaN,NaN,NaN,NaN,1861.0,NaN,London,NaN,...,Ontario,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5794.0,0
1,35.0,1826.0,NaN,NaN,NaN,NaN,1861.0,NaN,London,NaN,...,Ontario,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6103.0,0
2,46.0,1815.0,NaN,NaN,NaN,NaN,1861.0,NaN,London,NaN,...,Ontario,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14105.0,1
3,46.0,1815.0,NaN,NaN,NaN,NaN,1861.0,NaN,London,NaN,...,Ontario,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14108.0,1
4,0.0,1910.0,NaN,NaN,NaN,NaN,1910.0,M,DENVER,NaN,...,CO,NaN,NaN,NaN,NaN,"DENVER,DENVER,CO",39.739235799999996,-104.99,24942.0,2
5,0.0,1910.0,NaN,NaN,NaN,NaN,1910.0,B,DENVER,NaN,...,CO,NaN,NaN,NaN,NaN,"DENVER,DENVER,CO",39.739235799999996,-104.99,24943.0,2
6,22.0,1848.0,NaN,NaN,NaN,NaN,1870.0,W,ALBANY,NaN,...,NY,NaN,NaN,NaN,NaN,"3-WDCOHOES,ALBANY,NY",42.761464000000004,-73.6972,35752.0,3
7,22.0,1848.0,NaN,NaN,NaN,NaN,1870.0,M,ALBANY,NaN,...,NY,NaN,NaN,NaN,NaN,"3-WDCOHOES,ALBANY,NY",42.761464000000004,-73.6972,35753.0,3
8,28.0,1892.0,NaN,NaN,NaN,NaN,1920.0,B,KING,NaN,...,WA,NaN,NaN,NaN,NaN,"SEATTLE,KING,WA",47.6062,-122.33207079999998,39156.0,4
9,28.0,1892.0,NaN,NaN,NaN,NaN,1920.0,M,KING,NaN,...,WA,NaN,NaN,NaN,NaN,"SEATTLE,KING,WA",47.6062,-122.33207079999998,39157.0,4


In [104]:
df_result_rl_4.to_csv('duplications_4.csv')

### 7. Relax 'place' and 'county' and 'race'.

In [105]:
result_rl_5 = compare_vectors_rl[(compare_vectors_rl['first_name'] == 1.0) & 
                                 (compare_vectors_rl['last_name'] == 1.0) &
                                 (compare_vectors_rl['state'] == 1.0) &
                                 # (compare_vectors_rl['county'] == 1.0) &
                                 # (compare_vectors_rl['place'] == 1.0) &
                                 (compare_vectors_rl['sex'] == 1) &
                                 # (compare_vectors_rl['race'] == 1) &
                                 (compare_vectors_rl['census_year'] == 1) &
                                 (compare_vectors_rl['calculated_birth_year'] == 1)].\
                                 reset_index()
result_rl_5 = result_rl_5[result_rl_5['level_0'] != result_rl_5['level_1']].reset_index()
result_rl_5.drop('index', axis = 1, inplace = True)

In [106]:
result_rl_5['indexid'] = result_rl_5.apply(lambda row: create_indexid(row), axis = 1)
result_rl_5 = result_rl_5.drop_duplicates('indexid')

In [107]:
diff_5 = set(result_rl_5['indexid']).difference(set(result_rl['indexid']))

In [108]:
delt_5 = result_rl_5[result_rl_5['indexid'].isin(diff_5)]

In [109]:
delt_5

,level_0,level_1,first_name,last_name,state,county,place,race,sex,calculated_birth_year,census_year,indexid
0,4,14662,1.0,1.0,1.0,0.0,0.0,1,1,1,1,146624
2,14663,14694,1.0,1.0,1.0,0.0,0.0,1,1,1,1,1466314694
4,18796,18797,1.0,1.0,1.0,0.0,1.0,1,1,1,1,1879618797
6,23466,23467,1.0,1.0,1.0,0.0,0.0,1,1,1,1,2346623467
8,50395,50479,1.0,1.0,1.0,0.0,0.0,1,1,1,1,5039550479
...,...,...,...,...,...,...,...,...,...,...,...,...
1105,50675,50791,1.0,1.0,1.0,0.0,0.0,1,1,1,1,5067550791
1107,50676,50791,1.0,1.0,1.0,0.0,0.0,1,1,1,1,5067650791
1110,50682,50754,1.0,1.0,1.0,0.0,1.0,1,1,1,1,5068250754
1112,50701,50767,1.0,1.0,1.0,0.0,0.0,1,1,1,1,5070150767


In [111]:
df_result_rl_5 = pd.DataFrame()
for i in zip(delt_5['level_0'], delt_5['level_1']):
    df_result_rl_5 = df_result_rl_5.append(df.iloc[i[0]])
    df_result_rl_5 = df_result_rl_5.append(df.iloc[i[1]])   
df_result_rl_5.reset_index(inplace = True)
df_result_rl_5 = df_result_rl_5.drop('index', axis = 1)

In [112]:
num_list = []
for i in range(int(df_result_rl_5.shape[0]/2)):
    num_list.append(i)
    num_list.append(i)
df_result_rl_5['dup_pair'] = pd.Series(num_list)

In [113]:
df_result_rl_5

,Age,CalculatedBirthYear,Cannot Read,Cannot Write,Cause of Death,Census Year Match,Census.Year,Color..Race.or.Ethnicity,County_x,Date Mustered Out,...,State/Province_x,Total of Matches,WARD,Year of Immigration to Canada if an Immigrant,Year of this Record,address,lat_x,long_x,unique_id,dup_pair
0,48.0,1832.0,NaN,NaN,NaN,NaN,1880.0,B,ERIE,NaN,...,NY,NaN,NaN,NaN,NaN,"4-WD;BUFFALO,ERIE,NY",42.9023,-78.6338,5.0,0
1,48.0,1832.0,1.0,0.0,1.0,NaN,1880.0,Black,NaN,NaN,...,NY,NaN,0.0,3.0,NaN,6,NaN,5.90975e+09,14663.0,0
2,46.0,1834.0,0.0,0.0,4.0,NaN,1880.0,Black,NaN,NaN,...,NY,NaN,0.0,1.0,NaN,6,NaN,5.90975e+09,14664.0,1
3,46.0,1834.0,NaN,NaN,NaN,NaN,1880.0,B,ERIE,NaN,...,NY,NaN,NaN,NaN,NaN,"4-WD;BUFFALO,ERIE,NY",42.9023,-78.6338,14695.0,1
4,11.0,1869.0,0.0,0.0,4.0,NaN,1880.0,Black,NaN,NaN,...,IL,NaN,0.0,0.0,NaN,9,NaN,1.37068e+09,18797.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,30.0,1850.0,NaN,NaN,NaN,NaN,1880.0,B,SUFFOLK,NaN,...,MA,NaN,NaN,NaN,NaN,"BOSTON,SUFFOLK,MA",42.360082500000004,-71.0589,50755.0,442
886,33.0,1847.0,3.0,2.0,1.0,NaN,1880.0,Mulatto(BlackandWhite),NaN,NaN,...,NY,NaN,0.0,0.0,NaN,5,NaN,6090789000,50702.0,443
887,33.0,1847.0,NaN,NaN,NaN,NaN,1880.0,M,KINGS,NaN,...,NY,NaN,NaN,NaN,NaN,"19-WD;BROOKLYN,KINGS,NY",40.7271,-73.9368,50768.0,443
888,5.0,1875.0,0.0,0.0,3.0,NaN,1880.0,Mulatto(BlackandWhite),NaN,NaN,...,NY,NaN,0.0,2.0,NaN,5,NaN,6090789000,50718.0,444


In [114]:
df_result_rl_5.to_csv('duplications_5.csv')